#  **Hands-on-session1 (Querying data from materials databases)**

# **Necessary Libraries**

In [ ]:
!pip install matminer[citrine]
!pip install pyyaml
!pip install mp_api
!pip install pandas==2.2.2
!git clone https://github.com/RealBack1/practice.git

# **Practice**

**Getting Data**

*Revised from Joseph Montoya's notebook*

This notebook demonstrates a few basic examples from matminer's data retrieval features. Matminer supports data retrieval from the following sources.

*   Materials Project (https://materialsproject.org/)

This notebook was last updated 11/15/18 for version 0.4.5 of matminer.

Each resource has a corresponding object in matminer designed for retrieving data and preprocessing it into a pandas dataframe. In addition, matminer can also access and aggregate data from your own mongo database, if you have one.


**Data retrieval**

The materials project data retrieval tool, matminer.data_retrieval.retrieve_MP.MPDataRetrieval is initialized using an api_key that can be found on your personal dashboard page on materialsproject.org if you've created an account. If you've set your api key via pymatgen (e.g. pmg config --add PMG_MAPI_KEY YOUR_API_KEY_HERE), the data retrieval tool may be initialized without an input argument.

**We need our own MP API Key**

In [ ]:
## Materials Project API client: https://docs.materialsproject.org/downloading-data/using-the-api/getting-started
## Loading module of MP API client
from mp_api.client import MPRester
import pandas as pd

In [ ]:
## Put your own API key
my_api_key = ""
mpr = MPRester(my_api_key)

In [ ]:
list_of_available_fields = mpr.materials.summary.available_fields
print(list_of_available_fields)

**Practice 1: Get various properties of binary oxide materials with "AO" formula**

In [ ]:
docs = mpr.materials.summary.search(chemsys="*-O", formula="AB", fields=list_of_available_fields)
results = [doc.dict() for doc in docs]
df_full = pd.DataFrame(results)
df = df_full.loc[:, ["formula_pretty", "material_id", "formation_energy_per_atom", "energy_above_hull", "band_gap"]]
sdf = df.sort_values(by="formation_energy_per_atom", axis=0)
sdf.head(10)

**Practice 2: Draw the phase diagram for Li-Ni-O**

(This code was written with reference to the Jupyter notebook by Materials Virtual Lab(https://matgenb.materialsvirtuallab.org/))

In [ ]:
from pymatgen.analysis.phase_diagram import PhaseDiagram, PDPlotter
entries = mpr.get_entries_in_chemsys(elements=["Li", "Co", "O"], additional_criteria={"thermo_types": ["GGA_GGA+U"]})
phase = PhaseDiagram(entries)
plotter = PDPlotter(phase, backend="matplotlib")
plotter.show()

**Practice 3: Among the Li–X–O compounds, identify those with the R-3m crystal structure, and classify the stable ones (with energy_above_hull = 0)**

In [ ]:
pd.set_option("display.max_rows", None)   # None → 제한 없음
pd.set_option("display.max_columns", None)

docs = mpr.materials.summary.search(chemsys="Li-*-O", num_elements=3, spacegroup_symbol="R-3m", fields=list_of_available_fields)
results = [doc.dict() for doc in docs]
df_full = pd.DataFrame(results)
df = df_full.loc[:, ["formula_pretty", "material_id", "energy_above_hull","band_gap", "symmetry"]]
df.head()

In [ ]:
df

In [ ]:
df_zero = df[df["energy_above_hull"] == 0]
df_zero

**Practice 3-2: Among the LiXO2 compounds, obtain the structures of the most stable compounds (with energy_above_hull = 0)**

1.   항목 추가
2.   항목 추가



In [ ]:
docs = mpr.materials.summary.search(chemsys="Li-*-O", num_elements=3,formula="ABC2", energy_above_hull=(0,1e-6), fields=list_of_available_fields)
results = [doc.dict() for doc in docs]
df_full = pd.DataFrame(results)
df = df_full.loc[:, ["formula_pretty", "material_id", "energy_above_hull","band_gap", "symmetry"]]
df.head()